## Imports

In [12]:
import re
import copy
import pandas as pd
import signal
import os
import platform
import logging
import json
import requests
import base64
import sys
import time
import subprocess


## Init Java parser

In [13]:
s = [
    "java",
    "-jar",
    f"document-parser-1.6.7.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

parser_url = "http://localhost:8083/"
java_subprocess = None

docs = 'C:\\Users\\tyshchuk\\Downloads\\Трудовой_кодекс_Российской_Федерации_от_30_12_2001_N_197_ФЗ.docx'
try:
    i = 1
    try:
        print(f"Проверка, если парсер заупущен на {parser_url}")
        response = requests.post(
            f"{parser_url}status",
            headers=headers
        )
    except Exception as e:
        print(
            "Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс"
        )
        java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                           stdout=subprocess.PIPE, encoding="utf-8")
        time.sleep(2)
        while True:
            # time.sleep(0.1)
            output_log_spring = java_subprocess.stdout.readline()
            sys.stdout.write("\rПроверка соединения")
            sys.stdout.flush()
            i += 1
            if output_log_spring.find("Started DocumentParserService") != -1:
                print("\nГотово")
                java_subprocess.stdout.close()
                break
            if i > 31:
                raise Exception("Не удалось получить доступ к ранее запущенному парсеру")
    print("Запустился успешно")

    try:
        file = open(docs, 'rb')
        encoded_string = base64.b64encode(file.read())
        encoded_string = str(encoded_string)[2:-1]
    except Exception as e:
        print(f"при конвертации в base64, исключение = {e.msg}")
        print("="*200)

    doc_type = docs.split(".")[-1].upper()
    resArr = []
    document = []

    response = requests.post(
        f"{parser_url}document-parser",
        data=json.dumps({
            "base64Content": encoded_string,
            "documentFileType": doc_type
        }),
        headers=headers
    )
    try:
        resArr = response.json()['documents']
    except Exception as e:
        print(f"Ответ от парсера {response.json()}")
        print("="*200)

except Exception as e:
    logging.exception(e)
finally:
    #Смерть java процессу!
    if java_subprocess is not None:
        if platform.system() == 'Windows':
            subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
        elif platform.system() == 'Linux':
            os.kill(java_subprocess.pid, signal.SIGTERM)
        else:
            print('Не известная платформа, убейте в ручную процесс java')

Проверка, если парсер заупущен на http://localhost:8083/
Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс
Проверка соединения
Готово
Запустился успешно


In [14]:
resArr[0]['paragraphs'][6]['paragraphBody']['text']

'Целями трудового законодательства являются установление государственных гарантий трудовых прав и свобод граждан, создание благоприятных условий труда, защита прав и интересов работников и работодателей.\r\nОсновными задачами трудового законодательства являются создание необходимых правовых условий для достижения оптимального согласования интересов сторон трудовых отношений, интересов государства, а также правовое регулирование трудовых отношений и иных непосредственно связанных с ними отношений по:\r\nорганизации труда и управлению трудом;\r\nтрудоустройству у данного работодателя;\r\nподготовке и дополнительному профессиональному образованию работников непосредственно у данного работодателя;\r\n(в ред. Федерального закона от 02.07.2013 N 185-ФЗ)\r\nсоциальному партнерству, ведению коллективных переговоров, заключению коллективных договоров и соглашений;\r\nучастию работников и профессиональных союзов в установлении условий труда и применении трудового законодательства в предусмотренн

## Cleanup

In [15]:
note_cleanup_re: re.Pattern = re.compile(r'КонсультантПлюс\: примечание\.\n.*?\n', flags=re.MULTILINE)
ref_cleanup_re: re.Pattern = re.compile(r'\(.*N\s+\d+-ФЗ.*?\)\s')

for paragraph in resArr[0]['paragraphs']:
    paragraph['paragraphBody']['text'] = note_cleanup_re.sub('', paragraph['paragraphBody']['text'])
    paragraph['paragraphBody']['text'] = ref_cleanup_re.sub('', paragraph['paragraphBody']['text'])

## Splitting paragraphs

In [16]:
regex = re.compile('(\:)((.|\s)*?)(\.)')

In [17]:
buffer = []

for paragraph in resArr[0]['paragraphs'][3:]:
    findall: re.Pattern[str] = regex.findall(paragraph['paragraphBody']['text'])

    if len(findall) == 0:
        buffer.append(copy.deepcopy(paragraph))
    else:
        copy_paragraph = copy.deepcopy(paragraph)
        copy_paragraph['paragraphBody']['text'] = copy.copy(paragraph['paragraphBody']['text'].split(findall[0])[0])
        buffer.append(copy_paragraph)
        for text in findall:
            text = text[1]
            copy_paragraph = copy.deepcopy(paragraph)
            list_of_texts: str = text.split(';')
            for split_text in list_of_texts:
                copy_paragraph['paragraphBody']['text'] = copy.copy(split_text)
                buffer.append(copy_paragraph)
        if len(findall) > 1:
            copy_paragraph = copy.deepcopy(paragraph)
        copy_paragraph['paragraphBody']['text'] = copy.copy(paragraph['paragraphBody']['text'].split(findall[-1])[-1])
        buffer.append(copy_paragraph)

In [20]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
party = ''
section = ''
section2 = ''
chapter = ''
chapter2 = ''
# article = ''

df = pd.DataFrame(
    columns=['party', 'section_number', 'section_text', 'chapter_number', 'chapter_text', 'article_number',
             'article_text', 'text'])
for paragraph in resArr[0]['paragraphs'][3:]:
    if str(paragraph['paragraphHeader']['text']).startswith('ЧАСТЬ '):
        party = paragraph['paragraphHeader']['text']
        continue
    if str(paragraph['paragraphHeader']['text']).startswith('Раздел '):
        section, section2 = paragraph['paragraphHeader']['text'].split(". ", 1)

        continue
    if str(paragraph['paragraphHeader']['text']).startswith('Глава '):
        chapter, chapter2 = paragraph['paragraphHeader']['text'].split(". ", 1)
        continue
    if str(paragraph['paragraphHeader']['text']).startswith('Статья'):
        article = paragraph['paragraphHeader']['text'].split(". ", 1)
        df = df.append({
            'party': party,
            'section_number': section,
            'section_text': section2,
            'chapter_number': chapter,
            'chapter_text': chapter2,
            'article_number': article[0],
            'article_text': article[1],
            'text': paragraph['paragraphBody']['text']
        }, ignore_index=True)
        continue


In [42]:
df.head(10)

,party,section_number,section_text,chapter_number,chapter_text,article_number,article_text,text
0,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 1,Цели и задачи трудового законодательства,Целями трудового законодательства являются уст...
1,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 2,Основные принципы правового регулирования труд...,Исходя из общепризнанных принципов и норм межд...
2,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 3,Запрещение дискриминации в сфере труда,Каждый имеет равные возможности для реализации...
3,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 4,Запрещение принудительного труда,Принудительный труд запрещен.\r\nПринудительны...
4,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 5,"Трудовое законодательство и иные акты, содержа...",(в ред. Федерального закона от 30.06.2006 N 90...
...,...,...,...,...,...,...,...,...
522,ЧАСТЬ ШЕСТАЯ,Раздел XIV,ЗАКЛЮЧИТЕЛЬНЫЕ ПОЛОЖЕНИЯ,Глава 62,ОТВЕТСТВЕННОСТЬ ЗА НАРУШЕНИЕ\r\nТРУДОВОГО ЗАКО...,Статья 420,Сроки введения в действие настоящего Кодекса,Настоящий Кодекс вводится в действие с 1 февра...
523,ЧАСТЬ ШЕСТАЯ,Раздел XIV,ЗАКЛЮЧИТЕЛЬНЫЕ ПОЛОЖЕНИЯ,Глава 62,ОТВЕТСТВЕННОСТЬ ЗА НАРУШЕНИЕ\r\nТРУДОВОГО ЗАКО...,Статья 421,Порядок и сроки введения минимального размера ...,(в ред. Федерального закона от 20.04.2007 N 54...
524,ЧАСТЬ ШЕСТАЯ,Раздел XIV,ЗАКЛЮЧИТЕЛЬНЫЕ ПОЛОЖЕНИЯ,Глава 62,ОТВЕТСТВЕННОСТЬ ЗА НАРУШЕНИЕ\r\nТРУДОВОГО ЗАКО...,Статья 422,Признание утратившими силу отдельных законодат...,Признать утратившими силу с 1 февраля 2002 год...
525,ЧАСТЬ ШЕСТАЯ,Раздел XIV,ЗАКЛЮЧИТЕЛЬНЫЕ ПОЛОЖЕНИЯ,Глава 62,ОТВЕТСТВЕННОСТЬ ЗА НАРУШЕНИЕ\r\nТРУДОВОГО ЗАКО...,Статья 423,Применение законов и иных нормативных правовых...,Впредь до приведения законов и иных нормативны...


In [43]:
df[df['text'].duplicated(keep=False)]

,party,section_number,section_text,chapter_number,chapter_text,article_number,article_text,text
6,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 7,Утратила силу. - Федеральный закон от 30.06.20...,
99,ЧАСТЬ ТРЕТЬЯ,Раздел III,ТРУДОВОЙ ДОГОВОР,Глава 14,ЗАЩИТА ПЕРСОНАЛЬНЫХ ДАННЫХ РАБОТНИКА,Статья 85,Утратила силу. - Федеральный закон от 07.05.20...,
112,ЧАСТЬ ТРЕТЬЯ,Раздел IV,РАБОЧЕЕ ВРЕМЯ,Глава 15,ОБЩИЕ ПОЛОЖЕНИЯ,Статья 98,Утратила силу. - Федеральный закон от 30.06.20...,
192,ЧАСТЬ ТРЕТЬЯ,Раздел VII,ГАРАНТИИ И КОМПЕНСАЦИИ,Глава 26,"ГАРАНТИИ И КОМПЕНСАЦИИ РАБОТНИКАМ,\r\nСОВМЕЩАЮ...",Статья 175,Утратила силу с 1 сентября 2013 года. - Федера...,
446,ЧАСТЬ ЧЕТВЕРТАЯ,Раздел XII,ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА\r\nОТДЕЛЬНЫХ К...,Глава 55,ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА\r\nДРУГИХ КАТЕ...,Статья 351.2,Утратила силу с 1 января 2019 года. - Федераль...,
461,ЧАСТЬ ПЯТАЯ,Раздел XIII,ЗАЩИТА ТРУДОВЫХ ПРАВ И СВОБОД.\r\nРАССМОТРЕНИЕ...,Глава 57,ФЕДЕРАЛЬНЫЙ ГОСУДАРСТВЕННЫЙ КОНТРОЛЬ (НАДЗОР)\...,Статья 360,Утратила силу с 1 июля 2021 года. - Федеральны...,


In [44]:
with pd.ExcelWriter("Трудовой кодекс.xlsx", engine="openpyxl") as writer:
    df[~df['text'].duplicated(keep=False)].to_excel(writer, 'main', engine='xlsxwriter', index=False)
    sheets_good = writer.sheets['main']
    # sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])
    sheets_good.auto_filter.ref = sheets_good.dimensions

    logging.info("\nФайл создан")

PermissionError: [Errno 13] Permission denied: 'Трудовой кодекс.xlsx'